# От множества к пространству

Чем «пространство» отличается от «множества»? Если совсем на пальцах. Тем, что над его элементами определены операции. Соответственно, если мы просто «напишем» класс, то определим тип данных, элемент множества. А если реализуем для него операции — определим пространство.

Тяжело найти что-то, чего нет в стандартной библиотеке. А тем более во внешних...

![](https://imgs.xkcd.com/comics/python.png)

Давайте считать, что кольцо многочленов не реализовано. А ведь полезная вещь! А как его реализовать? А очень просто: [вот тут всё написано](https://docs.python.org/3.7/reference/datamodel.html#emulating-numeric-types)!

In [50]:
from numbers import Number
import itertools
import copy


class MatrixDomainError(ValueError):
    pass


class Matrix:
    """ """
    
    def __init__(self, arg: 'list[list[Number]]|Matrix'= 0):
        """
        
        """
    
        if isinstance(arg, Matrix):
            self.mat = copy.deepcopy(arg.mat)
        elif isinstance(arg, list):
            for i in range(1, len(arg)):
                if not isinstance(arg[i-1], list) and not isinstance(arg[i], list) and len(arg[i-1]) != len(arg[i]):
                    raise MatrixDomainError("You are trying to create matrix from " + repr(arg))
            self.mat = copy.deepcopy(arg)
        else:
            raise MatrixDomainError("You are trying to create matrix from " + repr(arg))

        self.cleanup(self.mat)

        
    @staticmethod
    def cleanup(mat: 'list[list[Number]]') -> 'None':
        epsilon = 1e-25
                    
        for i in range(len(mat)):
            for j in range(len(mat[i])):
                if abs(mat[i][j]) < epsilon:
                    mat[i][j] = 0
                    
        
    def __str__(self):
        return "[" + "\n ".join([row.__str__() for row in self.mat]) + "]"

    
    def __eq__(self, other: 'Matrix') -> 'bool':
        """Вот тут точно полезно почитать https://wiki.python.org/moin/MultipleDispatch..."""
        
        if isinstance(other, Matrix):
            return self.mat == other.mat
        else:
            raise MatrixDomainError("Can't say if Matrix is equal to " + str(type(other)))

    
    def __add__(self, other: 'Matrix') -> 'Matrix':
        if not isinstance(other, Matrix):
             raise MatrixDomainError("You can't add not a matrix " + str(type(other)))
                    
        if len(self.mat) != len(other.mat) or len(self.mat[0]) != len(other.mat[0]):
            raise MatrixDomainError("You can't add matrix of different size " + str(type(other)))
            
        res = copy.deepcopy(self.mat)
        for i in range(len(res)):
            for j in range(len(res[i])):
                res[i][j] += other.mat[i][j]
                    
        return Matrix(res)
        
        
    def __radd__(self, other: 'Matrix') -> 'Matrix':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'Matrix':
        return Matrix ([[-c for c in row] for row in self.mat])

    
    def __sub__(self, other: 'Matrix') -> 'Matrix':
        if not isinstance(other, Matrix):
            raise MatrixDomainError("You can't subtract not a matrix" + str(type(other)))

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other: 'Matrix') -> 'Matrix':
        return self.__neg__().__add__(other)

    
    def __mul__(self, a: 'Number') -> 'Matrix':
        if not isinstance(a, Number):
            raise MatrixDomainError("You can't multiply not on a number" + str(type(a)))
                    
        return Matrix ([[c*a for c in row] for row in self.mat])
        
        
    def __rmul__(self, other: 'Matrix') -> 'Matrix':
        return self.__mul__(other)  # Коммутативность


    


Ну а теперь давайте посмотрим его в деле!

In [60]:
m1 = Matrix([[1,2],[5,6]])
m2 = Matrix([[3,2],[0,4]])
m3 = Matrix([[4,5],[1,3]])
m4 = Matrix([[3,3],[4,6],[1,2]])

print(m1)
print(m2)
print(m3)

m15 = m1*5
print(m15)

m12 = m1 + m2
print(m12)

print(-m1)


[[1, 2]
 [5, 6]]
[[3, 2]
 [0, 4]]
[[4, 5]
 [1, 3]]
[[5, 10]
 [25, 30]]
[[4, 4]
 [5, 10]]
[[-1, -2]
 [-5, -6]]


## Задание

Придумать и реализовать какой-нибудь свой тип данных с операциями. Вот идеи:

* числа по модулю;
* обычные числа, но с ошибками при вычислениях;
* [кватеринионы](https://en.wikipedia.org/wiki/Quaternion);
* можно матрицы (в т.ч. вектора) — они и так в NumPy есть, но всё равно же интересно;
* да тот же многочлен, но при помощи [`defaultdict`](https://docs.python.org/3.7/library/collections.html#collections.defaultdict) — ему не повредит;
* что угодно на свой вкус, над чем можно определить арифметику или её подобие.